In [1]:
import os

In [2]:
from datetime import datetime

In [3]:
import pandas as pd

In [4]:
result_path = f'Результат/{datetime.now().strftime("%Y-%m-%d")}/'
if not os.path.exists(result_path):
    os.makedirs(result_path)

In [5]:
sbis_names = ["Дата",
            "Номер",
            "Сумма",
            "Статус",
            "Примечание",
            "Комментарий",
            "Контрагент",
            "ИНН/КПП",
            "Организация",
            "ИНН/КПП",
            "Тип документа",
            "Имя файла",
            "Дата",
            "Номер 1",
            "Сумма 1",
            "Сумма НДС",
            "Ответственный",
            "Подразделение",
            "Код",
            "Дата",
            "Время",
            "Тип пакета",
            "Идентификатор пакета",
            "Запущено в обработку",
            "Получено контрагентом",
            "Завершено",
            "Увеличение суммы",
            "НДC",
            "Уменьшение суммы",
            "НДС"
            ]

In [6]:
sbis_folder = "Входящие"
sbis_files = os.listdir(sbis_folder)

In [7]:
dfs = []
for file in sbis_files:
    if 'csv' not in file:
        continue
    df = pd.read_csv(sbis_folder + "/" + file, skiprows=1, sep = ";", encoding="windows-1251", header=None)
    dfs.append(df)

In [10]:
sbis = pd.concat(dfs, ignore_index=True)
sbis.columns = sbis_names
sbis.columns = [c.replace(' ', '_') for c in sbis.columns]

In [11]:
apteka_folder = "Аптеки/csv/correct/"
apteka_files = os.listdir(apteka_folder)

In [12]:
for file in apteka_files:

    if 'csv' not in file:
        continue

    apteka = pd.read_csv(apteka_folder + file, sep = ";", encoding="windows-1251")
    apteka["Номер счет-фактуры"] = ""
    apteka["Сумма счет-фактуры"] = ""
    apteka["Дата счет-фактуры"] = ""
    apteka["Сравнение дат"] = ""

    docs = ["СчФктр", "УпдДоп", "УпдСчфДоп", "ЭДОНакл"]

    for i, row in apteka.iterrows():
        nakl = row["Номер накладной"]

        if 'ЕАПТЕКА' in row["Поставщик"]:
            nakl += "/15"

        records = sbis[sbis.Номер.values == nakl]
        records = records[records.Тип_документа.isin(docs)]

        if records.empty:
            continue

        invoice = records.iloc[0]["Номер"]
        summ = records.iloc[0]["Сумма"]
        date = records.iloc[0]["Дата"][1]
        date = datetime.strptime(date, "%d.%m.%y").strftime("%d.%m.%Y")

        apteka.at[i, "Номер счет-фактуры"] = invoice
        apteka.at[i, "Сумма счет-фактуры"] = summ
        apteka.at[i, "Дата счет-фактуры"] = date
        apteka.at[i, "Сравнение дат"] = "" if (date == apteka.at[i, 'Дата накладной']) else "Не совпадает!"


    apteka_columns = ['№ п/п', 'Штрих-код партии', 'Наименование товара', 'Поставщик',
        'Дата приходного документа', 'Номер приходного документа',
        'Дата накладной', 'Номер накладной', 'Номер счет-фактуры',
        'Сумма счет-фактуры', 'Кол-во',
        'Сумма в закупочных ценах без НДС', 'Ставка НДС поставщика',
        'Сумма НДС', 'Сумма в закупочных ценах с НДС', 'Дата счет-фактуры', 'Сравнение дат']

    apteka = apteka[apteka_columns]
    apteka.to_excel(f"{result_path}{file.split('.csv')[0]} - результат.xlsx", index=False, encoding="windows-1251")
    print(f'{file} Обработан!')

C:\Users\Elena\AppData\Roaming\Python\Python311\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


366.csv Обработан!


C:\Users\Elena\AppData\Roaming\Python\Python311\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


А123.csv Обработан!
